The original code and notebook is aligned with a legacy version allennlp==0.9.0 (and spacy) - the new versions have non backward compatible APIs and requiers a more significant refactoring.

In [1]:
!pip install allennlp==0.9.0 spacy==2.1.4 overrides==3.1.0 -q # nbclient==0.5.13 -q
# !pip install allennlp==1.5.0 spacy overrides==3.1.0 -q

In [2]:
#warmup of spacy
from allennlp.common.util import get_spacy_model
try:
  get_spacy_model("en", pos_tags=False, parse=True, ner=False)
except:
  pass

In [3]:
# !rm -rf contextual_embedding_bias_measure/

In [4]:
# !git clone https://github.com/keitakurita/contextual_embedding_bias_measure.git
!git clone https://github.com/lleviraz/contextual_embedding_bias_measure.git
!cp -rf contextual_embedding_bias_measure/lib/ .

fatal: destination path 'contextual_embedding_bias_measure' already exists and is not an empty directory.


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
sys.path.append("lib")

In [7]:
import torch
import numpy as np
import pandas as pd
from typing import *
from pathlib import Path
from itertools import product
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import ttest_ind, ranksums
from mlxtend.evaluate import permutation_test
from bert_utils import Config, BertPreprocessor
from bias_utils import BiasUtils
%matplotlib inline

In [8]:
config = Config(
    model_type="bert-base-uncased",
    max_seq_len=128,
)

In [9]:
processor = BertPreprocessor(config.model_type, config.max_seq_len)

In [10]:
from pytorch_pretrained_bert import BertConfig, BertForMaskedLM
model = BertForMaskedLM.from_pretrained(config.model_type)
model.eval() # Important! Disable dropout
print()

In [11]:
bias_utils = BiasUtils(model,processor)

In [12]:
# def get_logits(sentence: str) -> np.ndarray:
#     return model(processor.to_bert_model_input(sentence))[0, :, :].cpu().detach().numpy()

# def softmax(arr, axis=1):
#     e = np.exp(arr)
#     return e / e.sum(axis=axis, keepdims=True)

# def get_mask_fill_logits(sentence: str, words: Iterable[str],
#                          use_last_mask=False, apply_softmax=True) -> Dict[str, float]:
#     mask_i = processor.get_index(sentence, "[MASK]", last=use_last_mask, accept_wordpiece=True)
#     logits = defaultdict(list)
#     out_logits = get_logits(sentence)
#     if apply_softmax: 
#         out_logits = softmax(out_logits)
#     return {w: out_logits[mask_i, processor.token_to_index(w, accept_wordpiece=True)] for w in words}

# def bias_score(sentence: str, gender_words: Iterable[Iterable[str]], 
#                word: str, gender_comes_first=True) -> Dict[str, float]:
#     """
#     Input a sentence of the form "GGG is XXX"
#     XXX is a placeholder for the target word
#     GGG is a placeholder for the gendered words (the subject)
#     We will predict the bias when filling in the gendered words and 
#     filling in the target word.
    
#     gender_comes_first: GGG comes before XXX
#     """
#     # e.g. probability of filling [MASK] with "he" vs. "she" when target is "programmer"
#     mwords, fwords = gender_words
#     all_words = mwords + fwords
#     subject_fill_logits = get_mask_fill_logits(
#         sentence.replace("XXX", word).replace("GGG", "[MASK]"), 
#         all_words, use_last_mask=not gender_comes_first,
#     )
#     subject_fill_bias = np.log(sum(subject_fill_logits[mw] for mw in mwords)) - \
#                         np.log(sum(subject_fill_logits[fw] for fw in fwords))
#     # male words are simply more likely than female words
#     # correct for this by masking the target word and measuring the prior probabilities
#     subject_fill_prior_logits = get_mask_fill_logits(
#         sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"), 
#         all_words, use_last_mask=gender_comes_first,
#     )
#     subject_fill_bias_prior_correction = \
#             np.log(sum(subject_fill_prior_logits[mw] for mw in mwords)) - \
#             np.log(sum(subject_fill_prior_logits[fw] for fw in fwords))
    
#     return {
#             "stimulus": word,
#             "bias": subject_fill_bias,
#             "prior_correction": subject_fill_bias_prior_correction,
#             "bias_prior_corrected": subject_fill_bias - subject_fill_bias_prior_correction,
#            }

In [13]:
bias_utils.get_mask_fill_logits("the [MASK] is beautiful", ["flower", "bug"])

{'bug': 1.0748338e-05, 'flower': 0.00074180553}

In [14]:
# def get_word_vector(sentence: str, word: str):
#     idx = processor.get_index(sentence, word, accept_wordpiece=True)
#     outputs = None
#     with torch.no_grad():
#         sequence_output, _ = model.bert(processor.to_bert_model_input(sentence),
#                                         output_all_encoded_layers=False)
#         sequence_output.squeeze_(0)
#     return sequence_output.detach().cpu().numpy()[idx]

In [15]:
# def cosine_similarity(x, y):
#     return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [16]:
# def get_effect_size(df1, df2, k="bias_prior_corrected"):
#     diff = (df1[k].mean() - df2[k].mean())
#     std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
#     return diff / std_

In [17]:
# def exact_mc_perm_test(xs, ys, nmc=100000):
#     n, k = len(xs), 0
#     diff = np.abs(np.mean(xs) - np.mean(ys))
#     zs = np.concatenate([xs, ys])
#     for j in range(nmc):
#         np.random.shuffle(zs)
#         k += diff < np.abs(np.mean(zs[:n]) - np.mean(zs[n:]))
#     return k / nmc

In [18]:
#sentence in index:706 from dreaddit train set
bias_utils.get_word_vector("daughter father abusive first however became left found two weeks later pregnant course begged pleaded second chance believed ", "abusive").shape

(768,)

In [19]:
rev_vocab = {v:k for k, v in processor.full_vocab.items()}

In [20]:
def to_words(wlist, filter_oov=True):
    return [w.strip() for w in wlist.lower().replace("\n", " ").split(", ") if w.strip() in rev_vocab or not filter_oov]

# Stress vs. Relaxed

In [21]:
k=20
female_words = list({'girl': 1.0, 'lot': 0.9322033898305084, 'mom': 0.9152542372881356, 'even': 0.8898305084745762, 'never': 0.8559322033898306, 'feel': 0.8220338983050848, 'much': 0.7796610169491526, 'mother': 0.7288135593220338, 'need': 0.7033898305084746, 'going': 0.6779661016949152, 'still': 0.5677966101694916, 'help': 0.5338983050847458, 'take': 0.5254237288135594, 'went': 0.5, 'back': 0.4915254237288136, 'make': 0.4830508474576271, 'well': 0.4745762711864407, 'point': 0.4661016949152542, 'p': 0.4576271186440678, 'woman': 0.4322033898305085, 'able': 0.423728813559322, 'wife': 0.423728813559322, 'times': 0.4152542372881356, 'sister': 0.4152542372881356, 'work': 0.4067796610169492}.keys())[:k]
male_words = list({'friend': 1.0, 'people': 0.6820809248554913, 'guy': 0.5260115606936416, 'family': 0.3786127167630058, 'person': 0.34971098265895956, 'love': 0.32947976878612717, 'good': 0.3092485549132948, 'life': 0.2976878612716763, 'someone': 0.28034682080924855, 'help': 0.2745664739884393, 'boyfriend': 0.2658959537572254, 'relationship': 0.2658959537572254, 'work': 0.23699421965317918, 'brother': 0.23121387283236994, 'anyone': 0.23121387283236994, 'man': 0.22832369942196531, 'dad': 0.22832369942196531, 'thought': 0.2254335260115607, 'old': 0.22254335260115607, 'kid': 0.2138728323699422, 'feel': 0.20809248554913296, 'father': 0.20809248554913296, 'new': 0.1994219653179191, 'thank': 0.1936416184971098, 'parent': 0.1936416184971098}.keys())[:k]
stress_words = list({'feel': 1.0, 'anxiety': 0.46534653465346537, 'feeling': 0.36386138613861385, 'trying': 0.349009900990099, 'bad': 0.3217821782178218, 'abuse': 0.297029702970297, 'hate': 0.2376237623762376, 'fear': 0.21534653465346534, 'need': 0.19306930693069307, 'someone': 0.1905940594059406, 'fucking': 0.1782178217821782, 'scared': 0.17574257425742573, 'us': 0.17574257425742573, 'panic': 0.17326732673267325, 'boyfriend': 0.16831683168316833, 'think': 0.16831683168316833, 'problem': 0.16584158415841585, 'attack': 0.16336633663366337, 'worse': 0.15594059405940594, 'anyone': 0.1485148514851485, 'angry': 0.14603960396039603, 'afraid': 0.14603960396039603, 'wrong': 0.14356435643564355, 'abusive': 0.14108910891089108, 'pain': 0.13861386138613863}.keys())[:k]
relaxed_words = list({'help': 1.0, 'good': 0.9748743718592965, 'started': 0.7135678391959799, 'well': 0.6834170854271356, 'able': 0.5376884422110553, 'made': 0.5276381909547738, 'together': 0.49246231155778897, 'thank': 0.48743718592964824, 'went': 0.4723618090452261, 'make': 0.3869346733668342, 'found': 0.3768844221105528, 'best': 0.37185929648241206, 'relationship': 0.35175879396984927, 'work': 0.34673366834170855, 'friend': 0.33668341708542715, 'used': 0.32663316582914576, 'first': 0.32160804020100503, 'another': 0.32160804020100503, 'helped': 0.32160804020100503, 'life': 0.3065326633165829, 'new': 0.3015075376884422, 'told': 0.3015075376884422, 'took': 0.2864321608040201, 'decided': 0.2814070351758794, 'though': 0.2814070351758794}.keys())[:k]

In [22]:
# def list_2_str(list):
#   return [','.join(list)]

for l in [female_words,male_words,stress_words,relaxed_words]:
  # print(list_2_str(l))
  print(l)


['girl', 'lot', 'mom', 'even', 'never', 'feel', 'much', 'mother', 'need', 'going', 'still', 'help', 'take', 'went', 'back', 'make', 'well', 'point', 'p', 'woman']
['friend', 'people', 'guy', 'family', 'person', 'love', 'good', 'life', 'someone', 'help', 'boyfriend', 'relationship', 'work', 'brother', 'anyone', 'man', 'dad', 'thought', 'old', 'kid']
['feel', 'anxiety', 'feeling', 'trying', 'bad', 'abuse', 'hate', 'fear', 'need', 'someone', 'fucking', 'scared', 'us', 'panic', 'boyfriend', 'think', 'problem', 'attack', 'worse', 'anyone']
['help', 'good', 'started', 'well', 'able', 'made', 'together', 'thank', 'went', 'make', 'found', 'best', 'relationship', 'work', 'friend', 'used', 'first', 'another', 'helped', 'life']


In [23]:
male_words = to_words("he, man, father, boy, brother")
female_words = to_words("she, woman, mother, girl, sister")
male_plural_words = to_words("boys, men, fathers, brothers")
female_plural_words = to_words("girls, women, mothers, sisters")


Since male words are simply more likely than female words, the code `bias_score` includes corrections for this by masking the target word and measuring the prior probabilities

In [24]:
def calc_bias_for_topic(topic_words_list):
  df1 = pd.concat([
      pd.DataFrame([bias_utils.bias_score("GGG likes XXX.", [male_words, female_words], w) for w in topic_words_list]),
      pd.DataFrame([bias_utils.bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in topic_words_list]),
      # pd.DataFrame([bias_utils.bias_score("GGG is interested in XXX.", [male_words, female_words], w) for w in topic_words_list])
      pd.DataFrame([bias_utils.bias_score("GGG is feeling XXX.", [male_words, female_words], w) for w in topic_words_list])
  ])
  return df1

In [25]:
df1 = calc_bias_for_topic(stress_words)
df1[-k:]

,stimulus,bias,prior_correction,bias_prior_corrected
0,feel,0.470788,0.294753,0.176036
1,anxiety,0.021616,0.294753,-0.273137
2,feeling,0.614186,0.294753,0.319434
3,trying,0.065343,0.294753,-0.229410
4,bad,-0.168169,0.294753,-0.462921
5,abuse,-0.294708,0.294753,-0.589460
6,hate,0.268701,0.294753,-0.026052
7,fear,0.201475,0.294753,-0.093278
8,need,0.182734,0.294753,-0.112018
9,someone,0.074385,0.294753,-0.220367


In [26]:
df1["bias_prior_corrected"].mean()

0.040313516890428856

In [28]:
df2 = calc_bias_for_topic(relaxed_words)
df2[-k:]

,stimulus,bias,prior_correction,bias_prior_corrected
0,help,-0.319322,0.294753,-0.614074
1,good,-0.159975,0.294753,-0.454728
2,started,0.226320,0.294753,-0.068433
3,well,-0.191389,0.294753,-0.486142
4,able,0.090212,0.294753,-0.204540
5,made,-0.326633,0.294753,-0.621386
6,together,-0.193437,0.294753,-0.488190
7,thank,0.052191,0.294753,-0.242562
8,went,0.105725,0.294753,-0.189028
9,make,-0.040059,0.294753,-0.334812


In [29]:
df2["bias_prior_corrected"].mean()

-0.15399659529319856

In [30]:
bias_utils.get_effect_size(df1, df2)

0.41601162434616906

In [31]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=2.320174525439556, pvalue=0.022049762070651367)

In [32]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=3.1754264805429417, pvalue=0.0014961642897455493)

In [33]:
bias_utils.exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"], )

0.02147